YOLOv5 🚀 by Ultralytics, AGPL-3.0 license


<br>
Run YOLOv5 detection inference on images, videos, directories, globs, YouTube, webcam, streams, etc.<br>
Usage - sources:<br>
    $ python detect.py --weights yolov5s.pt --source 0                               # webcam<br>
                                                     img.jpg                         # image<br>
                                                     vid.mp4                         # video<br>
                                                     screen                          # screenshot<br>
                                                     path/                           # directory<br>
                                                     list.txt                        # list of images<br>
                                                     list.streams                    # list of streams<br>
                                                     'path/*.jpg'                    # glob<br>
                                                     'https://youtu.be/LNwODJXcvt4'  # YouTube<br>
                                                     'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream<br>
Usage - formats:<br>
    $ python detect.py --weights yolov5s.pt                 # PyTorch<br>
                                 yolov5s.torchscript        # TorchScript<br>
                                 yolov5s.onnx               # ONNX Runtime or OpenCV DNN with --dnn<br>
                                 yolov5s_openvino_model     # OpenVINO<br>
                                 yolov5s.engine             # TensorRT<br>
                                 yolov5s.mlmodel            # CoreML (macOS-only)<br>
                                 yolov5s_saved_model        # TensorFlow SavedModel<br>
                                 yolov5s.pb                 # TensorFlow GraphDef<br>
                                 yolov5s.tflite             # TensorFlow Lite<br>
                                 yolov5s_edgetpu.tflite     # TensorFlow Edge TPU<br>
                                 yolov5s_paddle_model       # PaddlePaddle<br>


In [1]:
!pip install -qr requirements.txt comet_ml  # install

In [2]:
import argparse
import csv
import os
import platform
import sys
from pathlib import Path

In [3]:
import torch

In [4]:
# Obtener la ruta del directorio actual del cuaderno
NOTEBOOK_DIR = Path(os.path.abspath('')).resolve()

# Asegurarse de que la ruta del cuaderno esté en el PATH
if str(NOTEBOOK_DIR) not in sys.path:
    sys.path.append(str(NOTEBOOK_DIR))

# Establecer la variable ROOT como el directorio padre del cuaderno
ROOT = NOTEBOOK_DIR.parent

# Agregar la carpeta actual del cuaderno a ROOT de forma dinámica
current_folder_name = os.path.basename(NOTEBOOK_DIR)
ROOT = ROOT / current_folder_name

# Asegurarse de que ROOT esté en el PATH
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

# Convertir ROOT a una ruta relativa desde el directorio de trabajo actual
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))

# Imprimir el resultado para verificar
print("ROOT:", ROOT)


ROOT: .


In [5]:
from ultralytics.utils.plotting import Annotator, colors, save_one_box

In [6]:
from models.common import DetectMultiBackend
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadScreenshots, LoadStreams
from utils.general import (
    LOGGER,
    Profile,
    check_file,
    check_img_size,
    check_imshow,
    check_requirements,
    colorstr,
    cv2,
    increment_path,
    non_max_suppression,
    print_args,
    scale_boxes,
    strip_optimizer,
    xyxy2xywh,
)
from utils.torch_utils import select_device, smart_inference_mode

/home/ec2-user/SageMaker/elecnor/notebooks/ImagenesEPI/yolov5/utils/general.py


In [7]:
# @smart_inference_mode()
def run(
    weights=ROOT / "runs/train/bueno/best.pt",  # model path or triton URL
    source=ROOT / "../dataset/images/test",  # file/dir/URL/glob/screen/0(webcam)
    data=ROOT / "data/custom_dataset.yaml",  # dataset.yaml path
    imgsz=(640, 640),  # inference size (height, width)
    conf_thres=0.25,  # confidence threshold
    iou_thres=0.45,  # NMS IOU threshold
    max_det=1000,  # maximum detections per image
    device="",  # cuda device, i.e. 0 or 0,1,2,3 or cpu
    view_img=False,  # show results
    save_txt=False,  # save results to *.txt
    save_csv=True,  # save results in CSV format
    save_conf=False,  # save confidences in --save-txt labels
    save_crop=False,  # save cropped prediction boxes
    nosave=False,  # do not save images/videos
    classes=None,  # filter by class: --class 0, or --class 0 2 3
    agnostic_nms=False,  # class-agnostic NMS
    augment=False,  # augmented inference
    visualize=False,  # visualize features
    update=False,  # update all models
    project=ROOT / "runs/detect",  # save results to project/name
    name="exp",  # save results to project/name
    exist_ok=False,  # existing project/name ok, do not increment
    line_thickness=2,  # bounding box thickness (pixels)
    hide_labels=False,  # hide labels
    hide_conf=False,  # hide confidences
    half=False,  # use FP16 half-precision inference
    dnn=False,  # use OpenCV DNN for ONNX inference
    vid_stride=1,  # video frame-rate stride
):
    source = str(source)
    save_img = not nosave and not source.endswith(".txt")  # save inference images
    is_file = Path(source).suffix[1:] in (IMG_FORMATS + VID_FORMATS)
    is_url = source.lower().startswith(("rtsp://", "rtmp://", "http://", "https://"))
    webcam = source.isnumeric() or source.endswith(".streams") or (is_url and not is_file)
    screenshot = source.lower().startswith("screen")
    if is_url and is_file:
        source = check_file(source)  # download

    # Directories
    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
    (save_dir / "labels" if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Load model
    device = select_device(device)
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
    stride, names, pt = model.stride, model.names, model.pt
    imgsz = check_img_size(imgsz, s=stride)  # check image size

    # Dataloader
    bs = 1  # batch_size
    if webcam:
        view_img = check_imshow(warn=True)
        dataset = LoadStreams(source, img_size=imgsz, stride=stride, auto=pt, vid_stride=vid_stride)
        bs = len(dataset)
    elif screenshot:
        dataset = LoadScreenshots(source, img_size=imgsz, stride=stride, auto=pt)
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt, vid_stride=vid_stride)
    vid_path, vid_writer = [None] * bs, [None] * bs

    # Run inference
    model.warmup(imgsz=(1 if pt or model.triton else bs, 3, *imgsz))  # warmup
    seen, windows, dt = 0, [], (Profile(device=device), Profile(device=device), Profile(device=device))
    for path, im, im0s, vid_cap, s in dataset:
        with dt[0]:
            im = torch.from_numpy(im).to(model.device)
            im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
            im /= 255  # 0 - 255 to 0.0 - 1.0
            if len(im.shape) == 3:
                im = im[None]  # expand for batch dim
            if model.xml and im.shape[0] > 1:
                ims = torch.chunk(im, im.shape[0], 0)

        # Inference
        with dt[1]:
            visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
            if model.xml and im.shape[0] > 1:
                pred = None
                for image in ims:
                    if pred is None:
                        pred = model(image, augment=augment, visualize=visualize).unsqueeze(0)
                    else:
                        pred = torch.cat((pred, model(image, augment=augment, visualize=visualize).unsqueeze(0)), dim=0)
                pred = [pred, None]
            else:
                pred = model(im, augment=augment, visualize=visualize)
        # NMS
        with dt[2]:
            pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)

        # Second-stage classifier (optional)
        # pred = utils.general.apply_classifier(pred, classifier_model, im, im0s)

        # Define the path for the CSV file
        csv_path = save_dir / "predictions.csv"

        # Create or append to the CSV file
        def write_to_csv(image_name, prediction, confidence):
            """Writes prediction data for an image to a CSV file, appending if the file exists."""
            data = {"Image Name": image_name, "Prediction": prediction, "Confidence": confidence}
            with open(csv_path, mode="a", newline="") as f:
                writer = csv.DictWriter(f, fieldnames=data.keys())
                if not csv_path.is_file():
                    writer.writeheader()
                writer.writerow(data)

        # Process predictions
        for i, det in enumerate(pred):  # per image
            seen += 1
            if webcam:  # batch_size >= 1
                p, im0, frame = path[i], im0s[i].copy(), dataset.count
                s += f"{i}: "
            else:
                p, im0, frame = path, im0s.copy(), getattr(dataset, "frame", 0)
            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # im.jpg
            txt_path = str(save_dir / "labels" / p.stem) + ("" if dataset.mode == "image" else f"_{frame}")  # im.txt
            s += "%gx%g " % im.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            imc = im0.copy() if save_crop else im0  # for save_crop
            annotator = Annotator(im0, line_width=line_thickness, example=str(names))
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], im0.shape).round()
                # Print results
                for c in det[:, 5].unique():
                    n = (det[:, 5] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
                # Write results
                for *xyxy, conf, cls in reversed(det):
                    c = int(cls)  # integer class
                    label = names[c] if hide_conf else f"{names[c]}"
                    confidence = float(conf)
                    confidence_str = f"{confidence:.2f}"
                    if save_csv:
                        write_to_csv(p.name, label, confidence_str)
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                        with open(f"{txt_path}.txt", "a") as f:
                            f.write(("%g " * len(line)).rstrip() % line + "\n")
                    if save_img or save_crop or view_img:  # Add bbox to image
                        c = int(cls)  # integer class
                        label = None if hide_labels else (names[c] if hide_conf else f"{names[c]} {conf:.2f}")
                        annotator.box_label(xyxy, label, color=colors(c, True))
                    if save_crop:
                        save_one_box(xyxy, imc, file=save_dir / "crops" / names[c] / f"{p.stem}.jpg", BGR=True)

            # Stream results
            im0 = annotator.result()
            if view_img:
                if platform.system() == "Linux" and p not in windows:
                    windows.append(p)
                    cv2.namedWindow(str(p), cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)  # allow window resize (Linux)
                    cv2.resizeWindow(str(p), im0.shape[1], im0.shape[0])
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == "image":
                    cv2.imwrite(save_path, im0)
                else:  # 'video' or 'stream'
                    if vid_path[i] != save_path:  # new video
                        vid_path[i] = save_path
                        if isinstance(vid_writer[i], cv2.VideoWriter):
                            vid_writer[i].release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                        save_path = str(Path(save_path).with_suffix(".mp4"))  # force *.mp4 suffix on results videos
                        vid_writer[i] = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
                    vid_writer[i].write(im0)

        # Print time (inference-only)
        LOGGER.info(f"{s}{'' if len(det) else '(no detections), '}{dt[1].dt * 1E3:.1f}ms")

    # Print results
    t = tuple(x.t / seen * 1e3 for x in dt)  # speeds per image
    LOGGER.info(f"Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS per image at shape {(1, 3, *imgsz)}" % t)
    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ""
        LOGGER.info(f"Results saved to {colorstr('bold', save_dir)}{s}")
    if update:
        strip_optimizer(weights[0])  # update model (to fix SourceChangeWarning)

In [8]:
def main():
    """Executes YOLOv5 model inference with given options, checking requirements before running the model."""
    check_requirements(ROOT / "requirements.txt", exclude=("tensorboard", "thop"))
    run()

In [12]:
main()

YOLOv5 🚀 v7.0-294-gdb125a20 Python-3.10.13 torch-2.1.0 CPU

Fusing layers... 
Model summary: 157 layers, 1765930 parameters, 0 gradients, 4.1 GFLOPs
image 1/33 /home/ec2-user/SageMaker/elecnor/notebooks/ImagenesEPI/dataset/images/test/05398d7085eb502c20240215153352610.jpeg: 640x640 2 guantes, 1 casco, 1 bota, 1 arnes_pecho, 57.3ms
image 2/33 /home/ec2-user/SageMaker/elecnor/notebooks/ImagenesEPI/dataset/images/test/0d60ff04228816ea20240213064709967.jpeg: 640x640 1 casco, 2 botas, 53.7ms
image 3/33 /home/ec2-user/SageMaker/elecnor/notebooks/ImagenesEPI/dataset/images/test/167bc545e37e6ba420240215115640158.jpeg: 640x640 1 guante, 1 casco, 2 botas, 53.9ms
image 4/33 /home/ec2-user/SageMaker/elecnor/notebooks/ImagenesEPI/dataset/images/test/1db83e78239a1af420240215070444335.jpeg: 640x640 1 casco, 1 arnes_pecho, 1 arnes_espalda, 53.5ms
image 5/33 /home/ec2-user/SageMaker/elecnor/notebooks/ImagenesEPI/dataset/images/test/20e7956c569a2aea20240118152706169.jpeg: 640x640 1 casco, 1 bota, 53.3ms